## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# 參考 https://keras.io/zh/applications/

#model=keras.applications.xception.Xception(include_top=False, weights='imagenet',\
                                  #input_tensor=input_tensor,pooling=None, classes=10)


'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',\
                                  input_tensor=input_tensor,pooling=None, classes=10)


model.summary()


Using TensorFlow backend.
W1223 14:31:23.879504 17212 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1223 14:31:24.749957 17212 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1223 14:31:24.759960 17212 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1223 14:31:24.762954 17212 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1223 14:31:24.777909 17212 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\Tesseract\lib\

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
#x = GlobalAveragePooling2D(name='avg_pool')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.1)(x)





predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  
C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
W1223 14:31:30.622792 17212 deprecation.py:506] From C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model深度： 179


C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  del sys.path[0]


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=10)

W1223 14:31:33.218145 17212 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1223 14:31:33.303915 17212 deprecation.py:323] From C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
50000/50000 [==============================] - 163s 3ms/step - loss: 1.2885 - acc: 0.5801
Epoch 2/10
50000/50000 [==============================] - 158s 3ms/step - loss: 0.9935 - acc: 0.6629
Epoch 3/10
50000/50000 [==============================] - ETA: 0s - loss: 0.8798 - acc: 0.6997  ETA: 10s - loss: 0.8733 - - ETA: 8s - lo - ETA: 2s - - 156s 3ms/step - loss: 0.8799 - acc: 0.6997
Epoch 4/10
50000/50000 [==============================] - 154s 3ms/step - loss: 0.8361 - acc: 0.7218
Epoch 5/10
50000/50000 [==============================] - 157s 3ms/step - loss: 0.7718 - acc: 0.7370
Epoch 6/10
50000/50000 [==============================] - 156s 3ms/step - loss: 0.6821 - acc: 0.7573
Epoch 7/10
50000/50000 [==============================] - 156s 3ms/step - loss: 0.6047 - acc: 0.7884
Epoch 8/10
50000/50000 [==============================] - 155s 3ms/step - loss: 0.5301 - acc: 0.8125
Epoch 9/10
50000/50000 [==============================] - 154s 3ms/step - loss: 0.4807 - acc: 0.830